In [1]:
import numpy             as np
import matplotlib.pyplot as plt
import scipy.sparse      as spa

import sys; sys.path.insert(0, '..')

import hylife.utilitis_FEEC.basics.spline_evaluation_3d as eva_3d

import hylife.utilitis_FEEC.bsplines     as bsp 
import hylife.utilitis_FEEC.Bspline      as bspline
import hylife.utilitis_FEEC.spline_space as spl

import hylife.geometry.mappings_analytical    as map_ana

import hylife.geometry.mappings_discrete      as map_dis
import hylife.geometry.mappings_discrete_fast as map_dis_fast

import hylife.utilitis_FEEC.basics.mass_matrices_3d as mass

from scipy.sparse.linalg import splu

In [2]:
# setup
Nel        = [32, 32, 2]                    # number of elements on logical domain
bc         = [False, False, True]         # boundary conditions (True: periodic, False: else)
p          = [4, 4, 1]                     # spline degrees  

el_b       = [np.linspace(0., 1., Nel + 1) for Nel in Nel]                       # element boundaries
T          = [bsp.make_knots(el_b, p, bc) for el_b, p, bc in zip(el_b, p, bc)]   # knot vectors (for N functions)

nq_el      = [6, 6, 6]                     # number of quadrature points per element for integrations over whole domain

spaces     = [spl.spline_space_1d(T, p, bc, nq_el) for T, p, bc, nq_el in zip(T, p, bc, nq_el)] # 1d B-spline spaces
t_space    = spl.tensor_spline_space(spaces)

greville   = [bsp.greville(T, p, bc) for T, p, bc in zip(T, p, bc)]

N          = [bspline.Bspline(spl.T, spl.p)     for spl in spaces]     # N-splines for comparison
D          = [bspline.Bspline(spl.t, spl.p - 1) for spl in spaces]     # D-splines for comparison

In [3]:
# compute control points of discrete mapping
Lx    = 2.5
Ly    = 2.
Lz    = 1.5

x = lambda eta1, eta2, eta3 : Lx*eta1
y = lambda eta1, eta2, eta3 : Ly*eta2
z = lambda eta1, eta2, eta3 : Lz*eta3

kind_map   = 1
params_map = [Lx, Ly, Lz]

In [3]:
# compute control points of discrete mapping
Lx    = 2.
Ly    = 2.
Lz    = 1.5
alpha = 0.1

x = lambda eta1, eta2, eta3 : Lx*(eta1 + alpha*np.sin(2*np.pi*eta1)*np.sin(2*np.pi*eta2))
y = lambda eta1, eta2, eta3 : Ly*(eta2 + alpha*np.sin(2*np.pi*eta1)*np.sin(2*np.pi*eta2))
z = lambda eta1, eta2, eta3 : Lz*eta3

kind_map   = 3
params_map = [Lx, Ly, alpha, Lz]

In [3]:
# compute control points of discrete mapping
y0  = 0. 
eps = 0.3
e   = 1.4
xi  = 1/np.sqrt(1 - eps**2/4)
Lz    = 1.5

x = lambda eta1, eta2, eta3 : 1/eps*(1 - np.sqrt(1 + eps*(eps + 2*eta1*np.cos(2*np.pi*eta2))))
y = lambda eta1, eta2, eta3 : y0 + e*xi*eta1*np.sin(2*np.pi*eta2)/(1 + eps*x(eta1, eta2, eta3))
z = lambda eta1, eta2, eta3 : Lz*eta3

In [4]:
g1, g2, g3 = np.meshgrid(greville[0], greville[1], greville[2], indexing='ij')

N1 = spa.csc_matrix(bsp.collocation_matrix(T[0], p[0], greville[0], bc[0])) 
N2 = spa.csc_matrix(bsp.collocation_matrix(T[1], p[1], greville[1], bc[1]))
N3 = spa.csc_matrix(bsp.collocation_matrix(T[2], p[2], greville[2], bc[2]))
        
inter = splu(spa.kron(N1, spa.kron(N2, N3), format='csc'))

cx = inter.solve(x(g1, g2, g3).flatten()).reshape(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2])
cy = inter.solve(y(g1, g2, g3).flatten()).reshape(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2])
cz = inter.solve(z(g1, g2, g3).flatten()).reshape(t_space.NbaseN[0], t_space.NbaseN[1], t_space.NbaseN[2])

In [5]:
G = np.empty((3, 3), dtype=float)

In [6]:
eta1 = np.random.rand()
eta2 = np.random.rand()
eta3 = np.random.rand()

span_n1 = bsp.find_span(T[0], p[0], eta1)
span_n2 = bsp.find_span(T[1], p[1], eta2)
span_n3 = bsp.find_span(T[2], p[2], eta3)

span_n = [span_n1, span_n2, span_n3]

map_dis_fast.g(T[0], T[1], T[2], p, t_space.NbaseN, span_n, cx, cy, cz, eta1, eta2, eta3, G)

In [7]:
np.sqrt(np.linalg.det(G))

5.527881439960791

In [8]:
abs(map_dis_fast.det_df(T[0], T[1], T[2], p, t_space.NbaseN, span_n, cx, cy, cz, eta1, eta2, eta3))

5.527881439960791

In [10]:
abs(map_dis.det_df(T[0], T[1], T[2], p, t_space.NbaseN, cx, cy, cz, eta1, eta2, eta3))

5.527881439960789

In [5]:
M_ana = mass.mass_V0(t_space, 0, kind_map, params_map)

In [6]:
M_dis = mass.mass_V0(t_space, 1, tensor_space_F=t_space, cx=cx, cy=cy, cz=cz)

In [7]:
M_ana.max()

0.0005872448966378329

In [8]:
M_dis.max()

0.0005872449066085411

In [9]:
np.allclose(M_ana.toarray(), M_dis.toarray())

True